# Orientações gerais e informativos

### Secrets utilizados:
*   **HF_TOHEN** => Token do HuggingFace;
*   **GEMINI_API_KEY** => API KEY do Gemini.

### Contexto

Derivado do dataset "The AmazonTitles-1.3MM", a solução usaram um foundation model para:
- Receber perguntas com um contexto obtido por meio do arquivo "trn.json" que está contido dentro do dataset;
- A partir do prompt formado pela pergunta do usuário sobre o título do produdo, o modelo deverá gerar uma resposta baseada na pergunta do usuário trazendo como resultado do aprenzidado do fine-tuning os dados da sua descrição.

# Passo 0: Configuração

## Instalação das dependências

In [3]:
%pip install chromadb --quiet
#%pip install sentence-transformers --quiet
#%pip install -q -U google-generativeai --quiet
#%pip install transformers --quiet
#%pip install accelerate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.4 MB/s eta 0:00:00

## Download do CSV que será utilizado

Aqui há 3 possibilidades, a razão disso é que como a quantidade de registros é muito grande, foi gerado CSV de subconjunto desses registros:


*   Short: 10.000 registros;
*   Medium: 90.000 registros;
*   Full: 1.390.138 registros.

In [1]:
from pathlib import Path
from enum import Enum

class SizeData(Enum):
  SHORT = 1
  MEDIUM = 2
  FULL = 3


def download_csv(size_data: SizeData = SizeData.SHORT):
  if not Path('data/processed_data.csv').is_file():
    !mkdir data
    %cd data
    match(size_data):
      case SizeData.SHORT:
        !wget 'https://huggingface.co/datasets/DeyvidJLira/desafio-fiap-fase-3/resolve/main/processed_data_10000_rows.csv' -O processed_data.csv
      case SizeData.MEDIUM:
        !wget 'https://huggingface.co/datasets/DeyvidJLira/desafio-fiap-fase-3/resolve/main/processed_data_90000_rows.csv' -O processed_data.csv
      case SizeData.FULL:
        !wget 'https://huggingface.co/datasets/DeyvidJLira/desafio-fiap-fase-3/resolve/main/processed_data_1390138_rows.csv' -O processed_data.csv
    %cd ..
  else:
    print("Arquivo já existe")

download_csv(size_data=SizeData.SHORT)

/content/data
--2024-11-25 19:08:55--  https://huggingface.co/datasets/DeyvidJLira/desafio-fiap-fase-3/resolve/main/processed_data_10000_rows.csv
Resolving huggingface.co (huggingface.co)... 65.8.243.92, 65.8.243.46, 65.8.243.90, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.92|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/5b/ff/5bff399b96df2ff2e73b982b28f65cf1a5326a795a84d35495208d4eaa30ffd9/3b32388b58065a215dc53da4bd13163208cbceb12067ab87a603008ec96629b2?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27processed_data_10000_rows.csv%3B+filename%3D%22processed_data_10000_rows.csv%22%3B&response-content-type=text%2Fcsv&Expires=1732820935&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjgyMDkzNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzViL2ZmLzViZmYzOTliOTZkZjJmZjJlNzNiOTgyYjI4ZjY1Y2YxYTUzMjZhNzk1YTg0ZDM1NDk1MjA4ZDRlYWEzMGZmZDkvM2Iz

## Imports

In [4]:
from google.colab import userdata
import chromadb
from chromadb.config import Settings
import uuid;
from sentence_transformers import SentenceTransformer
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import google.generativeai as genai

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Criação das classes Database e Assistants que serão utilizadas durante o projeto

Nesta seção você encontrará 3 classes:
- Database: reune as operações destinadas ao VectorDB utilizado (ChromaDB);
- AssistantGeneric: reune as operações da aplicação que gerará o contexto e usará um módelo que esteja disponível no HugginFace (BERT, GPT e Llama) para construir uma resposta mais assertiva;
- AssistantGemini: reune as operações da aplicação que gerará o contexto e usará o Gemini para construir uma resposta mais assertiva.

Para o database foi utilizado o modelo de embedding **"paraphrase-MiniLM-L3-v2"** por ser mais eficiente para textos curtos. Antes utilizamos **"all-MiniLM-L6-v2"**, mas o resultado era ineficiente nas buscas que envolviam títulos com pouco conteúdo.

A fim de melhorar a perfomance, para o ChromeDB geramos um contexto e embeddamos ele, isso fez com que o retorno na busca de conteúdo relacionado ao título se torna-se mais precisa.

In [16]:
class Database:
    def __init__(self) -> None:
        self.client = chromadb.PersistentClient(path="./chromadb_data",
                                      settings=Settings(anonymized_telemetry=False))
        self.collection = self.client.get_or_create_collection("database")
        self.embedding_model = SentenceTransformer('paraphrase-MiniLM-L3-v2')


    def index_csv_data(self, csv_file, reindex: bool = False):
        if(reindex):
            self.collection.delete(where={"all": True})
        elif( self.collection.count() > 2):
            print("Já existem dados indexados, para uma nova indexação apague o chromadb_data")
            return

        df = pd.read_csv(
            csv_file,
            quotechar='"',
            delimiter=",",
            encoding="utf-8",
            on_bad_lines='warn'
        )

        for _, row in df.iterrows():
            doc_id = str(uuid.uuid4())

            title = row['title']
            content = row['content']
            context = f'Title: {title}\nContent: {content}'
            embedding = self.embedding_model.encode(context)
            self.collection.add(
                documents=[content],
                metadatas=[{"title": title, "id": doc_id}],
                embeddings=embedding,
                ids=[doc_id]
            )
        print("Dados indexados!")


    def query(self, query: str, top_k = 3):
        query_embedding = self.embedding_model.encode(query)
        return self.collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )



class AssistantGeneric:
    def __init__(self, model_name ) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            token=userdata.get('HF_TOKEN'))
        self.model = AutoModelForCausalLM.from_pretained(
            model_name,
            token=userdata.get('HF_TOKEN'),
            torch_dtype="float16",
            device_map="auto"
        )


    def query(self, message: str, db: Database) -> str:
        results = db.query(message)
        retrieved_docs = results['documents'][0]
        retrieved_titles = [doc['title'] for doc in results['metadatas'][0]]

        context = "\n\n".join([f'Title: {retrieved_titles[i]}\nContent: {retrieved_docs[i]}' for i in range(len(retrieved_docs))])
        prompt = f'With the Context: \n{context}\n\nAnswer concisely and accurately the Question: {message}.'

        inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = self.model.generate(inputs.input_ids, max_new_tokens=3)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)


class AssistantGemini:
    def __init__(self, api_key) -> None:
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel()


    def generate_context(self, message: str, db: Database) -> str:
        results = db.query(message)
        retrieved_docs = results['documents'][0]
        retrieved_titles = [doc['title'] for doc in results['metadatas'][0]]

        context = "\n\n".join([f'Title: {retrieved_titles[i]}\nContent: {retrieved_docs[i]}' for i in range(len(retrieved_docs))])
        return context


    def query(self, message: str, db: Database) -> str:
        results = db.query(message, top_k=10)
        retrieved_docs = results['documents'][0]
        retrieved_titles = [doc['title'] for doc in results['metadatas'][0]]

        context = "\n\n".join([f'Title: {retrieved_titles[i]}\nContent: {retrieved_docs[i]}' for i in range(len(retrieved_docs))])
        prompt = f'With the Context: \n{context}\n\nAnswer concisely and accurately the Question: {message}.'

        response = self.model.generate_content(prompt)
        return response.text


# Passo 1: Preparação

## Prepare Database

Os dados eles foram pré processados, onde ocorreu as seguintes operações em cima do "trn.json":
*   Gerou-se um dataframe;
```
import json
import gzip
import pandas as pd
```
```
data = None
with gzip.open('trn.json.gz') as f:
     data = [json.loads(line) for line in f]
```
```
df = pd.DataFrame(data)
df.to_csv("base_data.csv", index=False)
```
*   Removido todos os registros que não tinha conteúdo:
```
df2 = df.dropna()
```
*   Removido colunas desnecessárias:
```
df2 = df2.drop(columns=['target_ind', 'target_rel', 'uid'])
```
* Dados salvos em CSV:
```
df2.to_csv('processed_data.csv', index=False)
```
CSV disponível em: [Vide aqui](https://huggingface.co/datasets/DeyvidJLira/desafio-fiap-fase-3/blob/main/processed_data_1390138_rows.csv).

In [17]:
database = Database()
database.index_csv_data('/content/data/processed_data.csv', reindex=True)

Dados indexados!


## Prepare AI Assistant

In [14]:
assistant = AssistantGemini(userdata.get('GEMINI_API_KEY'))

Vamos ver o total de dados inseridos no banco

In [8]:
print(f'Total de registros inseridos: {database.collection.count()}')

Total de registros inseridos: 10000


Simulando como estar a obtenção de contexto

In [18]:
print(assistant.generate_context('Tell me about "Girls Ballet Tutu Neon Blue"', database))

Title: Girls Ballet Tutu Neon Blue
Content: Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun!

Title: Girls Ballet Tutu Neon Pink
Content: High quality 3 layer ballet tutu. 12 inches in length

Title: Ballerina Flying
Content: Geared to a younger audience, Alexa Brandenberg's Ballerina Flying brings readers to the narrator's Tuesday ballet class. "My name is Mina and I love to dance.... But I like ballet dance best of all./ To me, ballet dance is like flying." Watercolor vignettes depict her preparations; on double-page spreads, Miss Viola demonstrates positions on the verso page; opposite, her students practice. French pronunciations are provided.Copyright 2002 Cahners Business Information, Inc.


# Passo 2: Carregar Interface de Usuário

In [11]:
def ask_to_chatbot(question) -> str:
    try:
      return assistant.query(question, database)
    except Exception as e:
      return f"Ocorreu um erro: {e}"


from ipywidgets import widgets
from IPython.display import display, clear_output

prompt_input = widgets.Textarea(
    placeholder="Digite sua pergunta aqui (em inglês)...",
    description="Prompt:",
    layout=widgets.Layout(width="100%")
)

generate_button = widgets.Button(description="Enviar")
clear_prompt_button = widgets.Button(description="Limpar prompt")

output_area = widgets.Textarea(
    description="Answer:",
    disabled=True,
    layout=widgets.Layout(
        width="100%",
        height="100px",
        min_height="100px",
        overflow="auto"
    )
)

def on_button_click(b):
    prompt = prompt_input.value.strip()
    if prompt:
        output_area.value = ""
        answer = ask_to_chatbot(prompt)
        output_area.value = f"{answer}\n"
    else:
        output_area.value = "Prompt é necessário!"


def on_button_clear_prompt_click(b):
  prompt_input.value = ""


generate_button.on_click(on_button_click)
clear_prompt_button.on_click(on_button_clear_prompt_click)

display(prompt_input, output_area, generate_button, clear_prompt_button)



Textarea(value='', description='Prompt:', layout=Layout(width='100%'), placeholder='Digite sua pergunta aqui (…

Textarea(value='', description='Answer:', disabled=True, layout=Layout(height='100px', min_height='100px', ove…

Button(description='Enviar', style=ButtonStyle())

Button(description='Limpar prompt', style=ButtonStyle())